In [ ]:
!git clone https://github.com/tangqiaoyu/ToolAlpaca.git
!cd ToolAlpaca
!pip install -r /content/ToolAlpaca/requirements.txt
!pip install 'langchain==0.3.9'

Cloning into 'ToolAlpaca'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 98 (delta 16), reused 14 (delta 14), pack-reused 66 (from 1)
Receiving objects: 100% (98/98), 3.72 MiB | 10.28 MiB/s, done.
Resolving deltas: 100% (26/26), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jsonschema-spec to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import logging
from typing import List, Optional, Any, Dict

import requests
from langchain.chat_models.base import BaseChatModel
from langchain.schema import (
    BaseMessage,
    AIMessage,
    HumanMessage,
    SystemMessage,
    ChatResult,
    ChatGeneration,
)
from pydantic import PrivateAttr


# Define the custom LLM class
class Llama31ChatModel(BaseChatModel):
    api_key: str
    base_url: str
    model: str
    temperature: float = 0.5
    max_tokens: int = 3000
    logging_level: int = logging.INFO

    _logger: logging.Logger = PrivateAttr()

    class Config:
        arbitrary_types_allowed = True  # Allows arbitrary types like the logger

    def __init__(self, **data):
        super().__init__(**data)
        self._logger = logging.getLogger(__name__)
        logging.basicConfig(level=self.logging_level)

    @property
    def _llm_type(self) -> str:
        return "llama31"

    def _prepare_headers(self) -> Dict[str, str]:
        return {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

    def _prepare_context(self, messages: List[BaseMessage]) -> List[Dict[str, str]]:
        role_map = {
            HumanMessage: "user",
            AIMessage: "assistant",
            SystemMessage: "system"
        }

        return [{"role": role_map.get(type(message), "user"), "content": message.content} for message in messages]

    def _prepare_payload(
        self,
        context: List[Dict[str, str]],
        stop: Optional[List[str]] = None,
        **kwargs: Any
    ) -> Dict[str, Any]:
        payload = {
            "model": self.model,
            "messages": context,
            "temperature": kwargs.get("temperature", self.temperature),
            "max_tokens": kwargs.get("max_tokens", self.max_tokens),
        }
        if stop is not None:
            payload["stop"] = stop
        return payload

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> ChatResult:
        headers = self._prepare_headers()
        context = self._prepare_context(messages)
        payload = self._prepare_payload(context, stop, **kwargs)

        try:
            response = requests.post(
                f"{self.base_url}/chat/completions",
                headers=headers,
                json=payload,
            )
            response.raise_for_status()
            assistant_response = response.json()["choices"][0]["message"]["content"]
            self._logger.info("REQUEST: %s", payload)
            self._logger.info("RESPONSE: %s", assistant_response)
            ai_message = AIMessage(content=assistant_response)
            generation = ChatGeneration(message=ai_message)
            return ChatResult(generations=[generation])
        except requests.RequestException as e:
            self._logger.error("API request failed: %s", e)
            raise RuntimeError(f"API request failed: {e}")

In [ ]:
import json
import ast
import requests

from typing import Any, Dict
from pydantic import create_model
from langchain.tools import StructuredTool
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate


def create_tool_function(func_name: str, func_description: str, parameters: Dict[str, Any], outputs: Dict[str, Any], instances: list):
    def simulated_api(**kwargs):
        for instance in instances:
            if instance['input'] in kwargs.values():
                return instance['output']
        return "Simulated response based on input parameters."

    InputModel = create_model(f"{func_name}Input", **parameters)

    OutputModel = create_model(f"{func_name}Output", **outputs)

    simulated_api.__doc__ = func_description

    tool = StructuredTool.from_function(
        func=simulated_api,
        name=func_name,
        description=func_description,
        args_schema=InputModel,
        return_direct=True,
    )
    return tool

# Load the dataset
dataset_url = 'https://raw.githubusercontent.com/tangqiaoyu/ToolAlpaca/refs/heads/main/data/train_data.json'
response = requests.get(dataset_url)
data = json.loads(response.text)

api_data = data[1]

function_descriptions = api_data['Function_Description']
instances = api_data['Instances']

tools = []

for func_name, func_desc in function_descriptions.items():
    if func_name == 'components':
        continue

    lines = func_desc.split('\n')
    description = lines[0]
    parameters_line = [line for line in lines if line.startswith('Parameters:')][0]
    output_line = [line for line in lines if line.startswith('Output:')][0]

    # Extract parameters
    parameters_str = parameters_line.replace('Parameters:', '').strip()
    parameters_dict = ast.literal_eval(parameters_str)

    # Convert parameters to types
    parameters = {}
    for param_name, param_desc in parameters_dict.items():
        # Extract type from description
        type_str = param_desc.split('.')[1].strip().split('.')[0]
        if 'integer' in type_str:
            param_type = (int, ...)
        elif 'string' in type_str:
            param_type = (str, ...)
        elif 'Object' in type_str or 'object' in type_str:
            param_type = (Dict, ...)
        else:
            param_type = (Any, ...)
        parameters[param_name] = param_type

    # Extract outputs
    outputs = {'output': (Any, ...)}  # Simplify output for now

    # Create the tool
    tool = create_tool_function(
        func_name=func_name,
        func_description=description,
        parameters=parameters,
        outputs=outputs,
        instances=instances
    )
    tools.append(tool)

print(tools)

# Define your prompts as before
system_prompt = '''Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a JSON blob to specify a tool by providing an "action" key (tool name) and an "action_input" key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per JSON blob, as shown:

{{"action": $TOOL_NAME, "action_input": $INPUT}}

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action: $JSON_BLOB

Observation: action result
... (repeat Thought/Action/Observation N times)
Thought:
I know what to respond
Action:
{{"action": "Final Answer", "action_input": "Final response to human"}}


Begin! Reminder to ALWAYS respond with a valid JSON blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:

$JSON_BLOB

 then Observation'''

human_prompt = '''{input}
{agent_scratchpad}
(Reminder to respond in a JSON blob no matter what)'''



system_message = SystemMessagePromptTemplate.from_template(
    system_prompt,
    input_variables=["tools", "tool_names"],
)
human_message = HumanMessagePromptTemplate.from_template(
    human_prompt,
    input_variables=["input", "agent_scratchpad"],
)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        human_message,
    ]
)

# Initialize the custom LLM
llm = Llama31ChatModel(
    api_key="sk-or-vv-7fcc4ab944ca013feb7608fb7c0f001e5c12c32abf66233aad414183b4191a79",
    base_url="https://api.vsegpt.ru/v1",
    model="meta-llama/llama-3.1-70b-instruct",
    temperature=0.5,
    max_tokens=3000,
)

# Create the structured chat agent
agent = create_structured_chat_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
    stop_sequence=True,
)

# Create the AgentExecutor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    output_keys=["output"],
)

if __name__ == "__main__":
    # Use the instances as test cases
    for instance in instances:
        user_question = instance['input']
        expected_output = instance['output']

        response = agent_executor.invoke({"input": user_question})

        final_answer = response["output"]

        print(f"User Question: {user_question}")
        print(f"Agent's Response: {final_answer}")
        print(f"Expected Output: {expected_output}")
        print("-" * 50)


[StructuredTool(name='getCurrentWeather', description='Retrieve the current weather data for a specific location.', args_schema=<class '__main__.getCurrentWeatherInput'>, return_direct=True, func=<function create_tool_function.<locals>.simulated_api at 0x7dd8cefc0160>), StructuredTool(name='getHourlyForecast', description='Retrieve an hourly weather forecast for a specific location.', args_schema=<class '__main__.getHourlyForecastInput'>, return_direct=True, func=<function create_tool_function.<locals>.simulated_api at 0x7dd8cefc09d0>), StructuredTool(name='getDailyForecast', description='Retrieve a daily weather forecast for a specific location.', args_schema=<class '__main__.getDailyForecastInput'>, return_direct=True, func=<function create_tool_function.<locals>.simulated_api at 0x7dd8cefc1480>), StructuredTool(name='getHistoricalWeather', description='Retrieve historical weather data for a specific location and date range.', args_schema=<class '__main__.getHistoricalWeatherInput'>,

KeyboardInterrupt: 